# Julia の紹介2<br><small>Julia で Deep Learning</small>

<p style="text-align:center;font-size:150%;line-height:150%">2016/04/16 機械学習 名古屋 第3回勉強会<br>後藤 俊介 ( @antimon2 )</p>

## 自己紹介

+ 名前：後藤 俊介
+ 所属コミュニティ：**機械学習名古屋**（主催者の1人）、Python東海、Ruby東海
+ 言語：**Julia**, Python, Ruby, JavaScript, …
+ twitter: [@antimon2](https://twitter.com/antimon2 "あんちもん2(@antimon2)さん | Twitter")
+ Facebook: [antimon2](https://www.facebook.com/antimon2 "後藤 俊介")
+ GitHub: [antimon2](https://github.com/antimon2/ "antimon2 (GOTOH Shunsuke)")

↑今日もこの **Julia** の話。

## Julia とは？

※ 前回発表資料の流用（コピペ＋一部修正）です。

- [The Julia Language](http://julialang.org/)
- 2015/10/04 に [v0.4.0 がリリース](http://julialang.org/blog/2015/10/julia-0.4-release/)（2016/04/16 現在の最新は v0.4.5）
- Python/Ruby/R 等の「いいとこどり」言語（詳細後述）
- 動作が速い！（LLVM JIT コンパイル）

## Julia の特長

> + Rのように中身がぐちゃぐちゃでなく、
+ Rubyのように遅くなく、
+ Lispのように原始的またはエレファントでなく、
+ Prologのように変態的なところはなく、
+ Javaのように硬すぎることはなく、
+ Haskellのように抽象的すぎない
> 
> ほどよい言語である

引用：http://www.slideshare.net/Nikoriks/julia-28059489

Julia の目指すもの：

+ C のように高速だけど、  
  Ruby のような動的型付言語である
+ Lisp のように同じ文法で書けるマクロがあって、しかも  
  Matlab のような直感的な数式表現もできる
+ Python のように総合的なプログラミングができて、  
  R のように統計処理も得意で、  
  Perl のように文字列処理もできて、  
  Matlab のように線形代数もできて、  
  shell のように複数のプログラムを組み合わせることもできる
+ 超初心者にも習得は簡単で、  
  超上級者の満足にも応えられる
+ インタラクティブにも動作して、コンパイルもできる

（[Why We Created Julia](http://julialang.org/blog/2012/02/why-we-created-julia/) から抜粋・私訳）

## Deep Learning への適用

Julia の Deep Learning 用パッケージの紹介。

※ この記事は Julia v0.4.5 を基準としています。

+ [Mocha](https://github.com/pluskid/Mocha.jl)（C++ 用の [Caffe](http://caffe.berkeleyvision.org/) にインスパイアされて生まれたフレームワーク。互換性・可搬性・速度がウリ）
+ [MXNet](https://github.com/dmlc/MXNet.jl)（2015/10 に出たばかりの新しいフレームワーク。軽量・効率性・柔軟性がウリ）
+ [PyCall](https://github.com/stevengj/PyCall.jl)（Julia から Python を呼び出すパッケージ。Python にインストール済の機械学習パッケージ（例：TensorFlow）を利用可能）

## Mocha

+ すべて Julia で書かれた DeepLearning フレームワーク。
+ 考え方、記述方法等、色々 Caffe から引き継いでいる。
+ 扱えるデータ形式等、他のフレームワークとの互換性も持っている。

### Mocha のインストール

（Julia コンソールから↓）

In [ ]:
Pkg.add("Mocha")

※ `Blosc`, `HDF5`, `JLD` 等いくつかの依存パッケージも同時に追加される。

### 動作確認

In [ ]:
using Mocha

《※ Mocha を利用した、今回の 3LP サンプルの構築と学習・結果表示予定地》

## MXNet

+ Julia, Python, R, Go, JavaScript などに対応した DeepLearning フレームワーク。
+ 記述の簡潔さと、（それに伴う）「効率」と「柔軟性」の両立。
+ 処理のコア部分は C(C++) で記述されている（それにより軽量性と多言語対応を実現している）。

### MXNet のインストール

（Julia コンソールから↓）

In [ ]:
Pkg.add("MXNet")

※ ダイナミックリンクライブラリ libmxnet.so のビルドまでやってくれる。  
※ OpenCV の利用、CUDNN 等の利用に際しては、別途それらを有効にして libmxnet.so をビルドし、それを利用するよう `MXNet.jl` を再セットアップする必要あり。

### 動作確認

In [1]:
using MXNet

In [2]:
# 3LP ネットワーク構築
mlp = @mx.chain mx.Variable(:data)             =>
  mx.FullyConnected(name=:fc1, num_hidden=128) =>
  mx.Activation(name=:relu1, act_type=:relu)   =>
  mx.FullyConnected(name=:fc2, num_hidden=64)  =>
  mx.Activation(name=:relu2, act_type=:relu)   =>
  mx.FullyConnected(name=:fc3, num_hidden=10)  =>
  mx.SoftmaxOutput(name=:softmax)

MXNet.mx.SymbolicNode(MXNet.mx.MX_SymbolHandle(Ptr{Void} @0x00007fca8b195490))

In [3]:
# データ取得（データプロバイダ生成）
batch_size = 100
include(Pkg.dir("MXNet", "examples", "mnist", "mnist-data.jl"))
train_provider, eval_provider = get_mnist_providers(batch_size)

(MXNet.mx.MXDataProvider(MXNet.mx.MX_DataIterHandle(Ptr{Void} @0x00007fca8b27b160),Tuple{Symbol,Tuple}[(:data,(784,100))],Tuple{Symbol,Tuple}[(:softmax_label,(100,))],100,true,true),MXNet.mx.MXDataProvider(MXNet.mx.MX_DataIterHandle(Ptr{Void} @0x00007fca8b3851e0),Tuple{Symbol,Tuple}[(:data,(784,100))],Tuple{Symbol,Tuple}[(:softmax_label,(100,))],100,true,true))

In [4]:
# モデル構築・最適化

# モデル setup
model = mx.FeedForward(mlp, context=mx.cpu())

# optimization algorithm
optimizer = mx.SGD(lr=0.1, momentum=0.9)

# fit parameters
mx.fit(model, optimizer, train_provider, n_epoch=20, eval_data=eval_provider)

INFO: Start training on [CPU0]
INFO: Initializing parameters...
INFO: Creating KVStore...
INFO: Start training...
INFO: == Epoch 001 ==========
INFO: ## Training summary
INFO:           accuracy = 0.7548
INFO:               time = 1.2777 seconds
INFO: ## Validation summary
INFO:           accuracy = 0.9498
INFO: == Epoch 002 ==========
INFO: ## Training summary
INFO:           accuracy = 0.9575
INFO:               time = 0.8909 seconds
INFO: ## Validation summary
INFO:           accuracy = 0.9678
INFO: == Epoch 003 ==========
INFO: ## Training summary
INFO:           accuracy = 0.9700
INFO:               time = 0.9023 seconds
INFO: ## Validation summary
INFO:           accuracy = 0.9689
INFO: == Epoch 004 ==========
INFO: ## Training summary
INFO:           accuracy = 0.9760
INFO:               time = 0.8999 seconds
INFO: ## Validation summary
INFO:           accuracy = 0.9694
INFO: == Epoch 005 ==========
INFO: ## Training summary
INFO:           accuracy = 0.9811
INFO:               

In [5]:
# 予測
probs = mx.predict(model, eval_provider)

10x10000 Array{Float32,2}:
 1.06312e-10  5.80024e-13  2.965e-8     …  4.77526e-11  6.73342e-13
 7.46479e-8   1.50914e-11  0.999939        3.83869e-11  9.10979e-16
 5.39378e-9   1.0          2.33463e-5      2.85017e-13  2.81719e-13
 4.7356e-7    4.01732e-8   8.18917e-10     5.87185e-10  3.66529e-16
 8.03094e-10  2.75113e-16  3.04848e-6      3.1257e-14   2.58185e-13
 2.61006e-9   8.06292e-16  5.892e-9     …  0.999999     1.2713e-11 
 3.00019e-11  6.33867e-15  1.13649e-7      1.09904e-9   1.0        
 0.999999     6.59821e-11  3.33885e-5      2.07988e-12  6.45652e-19
 2.26256e-9   1.24287e-11  1.17842e-6      1.22676e-6   5.31528e-13
 1.94323e-7   3.90443e-21  7.36552e-9      7.82996e-11  1.12308e-19

In [6]:
# 予測精度確認

# collect all labels from eval data
labels = Array[]
for batch in eval_provider
    push!(labels, copy(mx.get(eval_provider, batch, :softmax_label)))
end
labels = cat(1, labels...)

# Now we use compute the accuracy
correct = 0
for i = 1:length(labels)
    # labels are 0...9
    if indmax(probs[:,i]) == labels[i]+1
        correct += 1
    end
end
accuracy = 100correct/length(labels)
println(mx.format("Accuracy on eval set: {1:.2f}%", accuracy))

Accuracy on eval set: 97.99%


## PyCall + TensorFlow

+ `PyCall` を利用すれば、Python にインストールした機械学習パッケージ等も利用可能（記述に独特のクセあり）。
+ 例として、`TensorFlow` を利用してみる。

### インストールと準備

In [ ]:
# 使用したいパッケージをインストールした Python の環境を環境変数に設定（pyenv や virtualenv で環境を分けている場合）
ENV["PYTHON"] = "/path/to/user_home/.pyenv/versions/2.7.11/envs/TensorFlow/bin/python"

# PyCall 本体のインストール
Pkg.add("PyCall")

# インストール済なら、依存ファイルを削除した上で再構築↓
# rm(Pkg.dir("PyCall","deps","PYTHON"))
# Pkg.build("PyCall")

### 動作確認

In [7]:
using PyCall

In [8]:
@pyimport tensorflow as tf
# ↑Python の import 文と同様の書き方ができる。

In [9]:
# データ取得
@pyimport tensorflow.examples.tutorials.mnist.input_data as input_data
mnist = input_data.read_data_sets("../MNIST_data/", one_hot=true)

Extracting ../MNIST_data/train-images-idx3-ubyte.gz
Extracting ../MNIST_data/train-labels-idx1-ubyte.gz
Extracting ../MNIST_data/t10k-images-idx3-ubyte.gz
Extracting ../MNIST_data/t10k-labels-idx1-ubyte.gz


PyObject <tensorflow.examples.tutorials.mnist.input_data.DataSets object at 0x347d7d150>

Python で言う `from A.B import C` ということをしたい場合は、`@pyimport A.B.C as C` としなければならない。

In [10]:
x = tf.placeholder(tf.float32, [nothing, 784])
y_ = tf.placeholder(tf.float32, [nothing, 10])

PyObject <tensorflow.python.framework.ops.Tensor object at 0x347dab550>

In [11]:
# 3LP 構築
W1 = tf.Variable(tf.random_normal(Int32[784, 128], mean=0.0, stddev=0.05))
b1 = tf.Variable(tf.zeros(Int32[128]))
W2 = tf.Variable(tf.random_normal(Int32[128, 64], mean=0.0, stddev=0.05))
b2 = tf.Variable(tf.zeros(Int32[64]))
W3 = tf.Variable(tf.random_normal(Int32[64, 10], mean=0.0, stddev=0.05))
b3 = tf.Variable(tf.zeros(Int32[10]))

h1 = tf.nn[:relu](tf.add(tf.matmul(x,  W1), b1))
h2 = tf.nn[:relu](tf.add(tf.matmul(h1, W2), b2))
y  = tf.nn[:softmax](tf.add(tf.matmul(h2, W3), b3))

PyObject <tensorflow.python.framework.ops.Tensor object at 0x348010050>

In [12]:
cross_entropy = tf.neg(tf.reduce_sum(tf.mul(y_, tf.log(y))))

PyObject <tensorflow.python.framework.ops.Tensor object at 0x34801e250>

In [13]:
optimizer = tf.train[:GradientDescentOptimizer](0.01)
train_step = optimizer[:minimize](cross_entropy)

PyObject <tensorflow.python.framework.ops.Operation object at 0x348097390>

In [14]:
tf_init = tf.initialize_all_variables()

PyObject <tensorflow.python.framework.ops.Operation object at 0x348097490>

In [15]:
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

PyObject <tensorflow.python.framework.ops.Tensor object at 0x3480c0f90>

In [16]:
sess = tf.Session()

sess[:run](tf_init)

for i in 1:2000
    batch_xs, batch_ys = mnist[:train][:next_batch](100)
    sess[:run](train_step, feed_dict=Dict(x => batch_xs, y_ => batch_ys))
end

println("accuracy:$(sess[:run](accuracy, feed_dict=Dict(x => mnist[:test][:images], y_ => mnist[:test][:labels])))")

accuracy:0.9495999813079834


I tensorflow/core/common_runtime/local_device.cc:40] Local device intra op parallelism threads: 4
I tensorflow/core/common_runtime/direct_session.cc:58] Direct session inter op parallelism threads: 4


## 参考

+ [The Julia Language](http://julialang.org/)（本家サイト、英語）
+ [Mocha](https://github.com/pluskid/Mocha.jl)
+ [MXNet](https://github.com/dmlc/MXNet.jl)
+ [PyCall](https://github.com/stevengj/PyCall.jl)
+ [Julia - josephmisiti/awesome-machine-learning](https://github.com/josephmisiti/awesome-machine-learning#julia-general-purpose)（Julia の機械学習関連ライブラリのリンクまとめ。英語）

ご清聴ありがとうございます。